In [1]:
# auto-compeletion faster
%config Completer.use_jedi = False

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
# from lightgbm import LGBMClassifier
from os.path import join as PJ

In [3]:
DATA = PJ("data",'hashed_feature.csv')
SEED = 1

# 資料簡介

這次的示範資料集是從Kaggle上2013年的[Amazon員工訪問權限預測挑戰賽](https://www.kaggle.com/c/amazon-employee-access-challenge)中取得
這個資料集，該資料集收集了Amazon公司中各個員工針對每個資源(例如網頁的logging)的訪問紀錄，當員工屬於能夠取得訪問權限時，系統卻不給訪問，又要向上申請才能取得權限，一來一往浪費的非常多時間，因此這場比賽希望能夠建構模型，減少員工訪問權限所需的人工流程，我們取出5個特徵如下 :


* Feature (X)

> RESOURCE : 資源ID

> MGR_ID : 員工主管的ID 

> ROLE_FAMILY_DESC : 員工類別擴展描述 (例如 軟體工程的零售經理)

> ROLE_FAMILY : 員工類別 (例如 零售經理)

> ROLE_CODE : 員工角色編碼 (例如 經理)

* Target (Y)

> ACTION : 

 >> 1 : RESOURCE 訪問權限取得
 
 >> 0 : RESOURCE 禁止訪問

In [4]:
train = pd.read_csv(DATA)
y = train['ACTION']
train = train[['RESOURCE', 'MGR_ID',
               'ROLE_FAMILY_DESC', 'ROLE_FAMILY',
               'ROLE_CODE']]
display(train.dtypes,
       train.shape,
       train.head(),
       'TARGET',
       y.value_counts())
train.info()
X_train, X_val, y_train, y_val = train_test_split(train, y, 
                                                  test_size=0.2,
                                                  random_state=SEED
                                                  , 
                                                  stratify = y)
print(
    'CARDINALITY of RESOURCE (ALL): ',
    len(train['RESOURCE'].unique()),
    'CARDINALITY of RESOURCE (Train): ',
    len(X_train['RESOURCE'].unique()),
    sep='\n'
    
)
print()

print('RESOURCE UNSEEN : ')
unseen = ~ X_train['RESOURCE'].isin(X_val['RESOURCE'])
unseen_resource = X_train['RESOURCE'][unseen].unique()

print(unseen_resource, 'num of resource unseen', len(unseen_resource), sep='\n')

RESOURCE            int64
MGR_ID              int64
ROLE_FAMILY_DESC    int64
ROLE_FAMILY         int64
ROLE_CODE           int64
dtype: object

(32769, 5)

,RESOURCE,MGR_ID,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,39353,85475,117906,290919,117908
1,17183,1540,118536,308574,118539
2,36724,14457,267952,19721,117880
3,36135,5396,240983,290919,118322
4,42680,5905,123932,19793,119325


'TARGET'

1    30872
0     1897
Name: ACTION, dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32769 entries, 0 to 32768
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   RESOURCE          32769 non-null  int64
 1   MGR_ID            32769 non-null  int64
 2   ROLE_FAMILY_DESC  32769 non-null  int64
 3   ROLE_FAMILY       32769 non-null  int64
 4   ROLE_CODE         32769 non-null  int64
dtypes: int64(5)
memory usage: 1.3 MB
CARDINALITY of RESOURCE (ALL): 
7518
CARDINALITY of RESOURCE (Train): 
6670

RESOURCE UNSEEN : 
[39151 39418 80307 ... 73956 86242 42897]
num of resource unseen
4625


# 問題描述

* 詞彙表不完整(out-of-vocabulary)
    * 當分類變數是嬰兒出生的 `hospital_id`, 使用者 `cookie_id`, 文章url `url-http_endpoint`，測試集中可能根本沒有你的`id`
* 高基數特徵造成的特徵列數大
    * 幾千萬個到幾百萬個特徵，導致訓練出來的模型很大
* 冷啟動
    * 新的文章 url-http_endpoint, 新的使用者 cookie
    
**Note** : 即使是 Ont-Hot-Encoding，你也應該保持 OOV 為 all zeros


# 解法

**Hashing**

1. 使用必然性(沒有隨機種子，沒有加鹽(salt?))，且可移植(同個算法可以同時用來訓練和服務)，的雜湊演算法

2. 推薦使用 Farmhash，他在BigQuerySQL中有實現

**Note** Farmhash 是一種必然性、分佈均勻、且許多程式語言都可以實作的雜湊演算法之一 - [farmhash](https://github.com/google/farmhash)

3. 我們可以簡單的label encoding之後，把各個數字 bucketize

# 為什麼有用

## OOV

1. 即使有不再Training set的 `RESOURCE`，他也會被分到某個`RESOURCE_bucket_id`中
    * 在模型服務時不會壞掉
    * 收到未知的`RESOURCE`時，會近乎隨機產生產生已經見過的`RESOURCE_bucket_id`，這個直當然不會是準確的，但他會是合理的值(也就是和她同一個`RESOURCE_bucket_id`)的行為
    
2. 你需要平衡 **合理地處理不再詞彙表內的輸入**、**讓模型準確地反應輸入**之間的需求
    * 根據經驗，比較好的做法大概是讓每個`bucket`有5筆資料
    
## High Cardinality

1. 只要選擇夠少的bucket數量，此問題就能獲得解決，從而維持系統記憶體的使用量以及模型大小的需求
2. 既然high cardinality原本解不了(無法訓練或是佔用過多資源)，那麼有損的編碼就會是一個可接受的折衷方案


## 冷啟動

1. 新的`RESOURCE`加入時，預測會比較不準，但隨著新的`RESOURCE`在新資料越出現越多，此模型的預測就會開始考量到新的`RESOURCE`(因為label encoding)

In [5]:
class Hasher:
    
    def __init__(self, n_buckets):
        self.n_buckets = n_buckets
        self.num_in_bucket = None
        self.value_to_label = {}
    
    def fit(self, df : pd.DataFrame, col) -> None:
        pass
    
    def fit_transform(self, df : pd.DataFrame,
                  col : str,
                  verbose : bool = True) -> pd.DataFrame:
        
        ####### do label encoding for original feature column #######
        res_df = df.copy()
        for label, value in enumerate(res_df[col].unique()):
            self.value_to_label[value] = label
            
        self.num_in_bucket = len(self.value_to_label.keys()) // self.n_buckets
        res_df[f'{col}_labels'] = res_df[col].map(self.value_to_label)
        res_df[f'{col}_bucket_id'] = (
            np.floor(res_df[f'{col}_labels'] / self.num_in_bucket).astype('int')
        )
        
        if verbose:
            print('original cardinality : ',len(self.value_to_label.keys()))
            print(f'hash in {self.n_buckets} buckets')
            print(f'{self.num_in_bucket} per buckets in general')
        
        return res_df
        


In [6]:
reosurce_hasher = Hasher(n_buckets=5000)

X_train_hashed = reosurce_hasher.fit_transform(X_train, col='RESOURCE')
X_train_hashed.head()

original cardinality :  6670
hash in 5000 buckets
1 per buckets in general


,RESOURCE,MGR_ID,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,RESOURCE_labels,RESOURCE_bucket_id
17261,77500,7459,135104,118870,125753,0,0
16696,79121,225257,311236,118424,118425,1,1
5664,20364,22504,128788,118424,120346,2,2
4784,33146,1728,128634,118398,121529,3,3
28797,39151,54684,279443,308574,118779,4,4


# 代價

1. 衝突 - 和 OneHotEncoding 比起來，hashing的特徵數較少，我們犧牲的準確表示資料的能力，就算你把`bucket` 的數量家大很多倍，衝突的機率還是會存在，除非你可以容許多個分類輸入共用一個雜湊值，否則就不要使用Hashed Features

2. 傾斜 - 當分類輸入高度傾斜時，準確度的損失就會特別嚴重
    * 考慮一個`Bucket`裡面有超多個`RESOURCE` id，那麼這些 id 的資料行為就會被一起計算，就像是某中不應該存在的平均
    
# 其他方案

1. 使用聚合特徵，例如引入TargetEncoding，變成該`RESOURCE`能取得訪問權限的機率，當然，Testing set的unseen `RESOURCE` 就會是類似平均值的處理方式，這確實也有類似效果


In [7]:
CARDINALITY = 6670

def calc_collision_prob(num_total, num_hash):
    no_collision_prob = 1.0
    for i in range(num_total):
        # i of the previous buckets is occupied now
        collision_likelihood = float(i) / num_hash
        no_collision_prob *= (1 - collision_likelihood)
    return 1 - no_collision_prob


data = []
for num_hash in [3, 10, 100, 1000, 5000, 100000, 200000, 2000000000]:
    data.append([num_hash, 
                 CARDINALITY/num_hash, 
                 calc_collision_prob(CARDINALITY, num_hash)
                ])
prob = pd.DataFrame(data, columns=['num_hash_buckets', 'entries_per_bucket', 'collision_prob'])
prob

,num_hash_buckets,entries_per_bucket,collision_prob
0,3,2223.333333,1.000000
1,10,667.000000,1.000000
2,100,66.700000,1.000000
3,1000,6.670000,1.000000
4,5000,1.334000,1.000000
5,100000,0.066700,1.000000
6,200000,0.033350,1.000000
7,2000000000,0.000003,0.011059
